In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

In [ ]:
az.style.use('arviz-white')
%config Completer.use_jedi = False
np.random.seed(1846)

In [ ]:
X = np.random.uniform(low=0, high=1.0, size=(100, 10))
f_x = 10*np.sin(np.pi*X[:,0]*X[:,1]) + 20*(X[:,2]-0.5)**2 + 10*X[:,3] + 5*X[:,4]
Y = np.random.normal(f_x, 1)

In [ ]:
alphas = [0.1, 0.25, 0.5]
idatas_at = {'10':{'0.1':{}, '0.25':{}, '0.5':{}},
             '20':{'0.1':{}, '0.25':{}, '0.5':{}},
             '50':{'0.1':{}, '0.25':{}, '0.5':{}},
             '100':{'0.1':{}, '0.25':{}, '0.5':{}},
             '200':{'0.1':{}, '0.25':{}, '0.5':{}}}
trees = [10, 20, 50, 100, 200]

for m in trees:
    for alpha in alphas:
        with pm.Model(rng_seeder=678) as model:
            μ = pm.BART('μ', X, Y, m=m, alpha=alpha) 
            σ = pm.HalfNormal('σ', 1)
            y = pm.Normal('y', μ, σ, observed=Y)
            idata = pm.sample(chains=4, random_seed=678)
            idatas_at[str(m)][str(alpha)] = idata

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(10, 3), sharey=True)

axes = axes.ravel()

for m, ax in zip(trees, axes):
    means = [idatas_at[str(m)][str(alpha)]['posterior']['μ'].mean(("chain", "draw")) for alpha in alphas]
    box = ax.boxplot(means, notch=True,
                     patch_artist=True, widths=0.5,
                     labels=alphas,
                     medianprops=dict(color="k"))
    for patch, color in zip(box['boxes'], ["C0", "C1", "C2", "C3"]):
        patch.set_facecolor(color)
        ax.set_title(f'm = {m}')

fig.supxlabel(fr'α', fontsize=16)
fig.supylabel(fr'μ', fontsize=16,
              rotation='horizontal')

plt.savefig('boxplots-alphas.png',
            dpi=300)